In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip uninstall tensorflow

Uninstalling tensorflow-2.2.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.2.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y

  Successfully uninstalled tensorflow-2.2.0


In [ ]:
!pip install tensorflow==1.14.0

     |████████████████████████████████| 109.2MB 87kB/s 
     |████████████████████████████████| 3.2MB 37.9MB/s 
     |████████████████████████████████| 491kB 44.3MB/s 
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0


In [ ]:
!pip install kashgari==1.1.4


In [ ]:
import os 
path="/content/drive/My Drive/Colab Notebooks/NLP/Project/"
os.chdir(path)

In [ ]:
import pandas as pd
import kashgari

kashgari.config.use_cudnn_cell = True # 使用GPU
train = pd.read_json("train_clean.txt")
train_x, train_y = train["text_clean"].tolist(), train["anno_text3"].tolist()
dev = pd.read_json("dev_clean.txt")
dev_x, dev_y = dev["text_clean"].tolist(), dev["anno_text3"].tolist()
test = pd.read_json("test_clean.txt")
test_x, test_y = test["text_clean"].tolist(), test["anno_text3"].tolist()

train.head()
print(type(train_x), type(train_y))
print(len(train_x),len(train_y),len(dev_x),len(dev_y), len(test_x), len(test_y))


<class 'list'> <class 'list'>
4164 4164 311 311 787 787


In [ ]:
from kashgari.embeddings import BERTEmbedding

bert_embed = BERTEmbedding('chinese_L-12_H-768_A-12', task=kashgari.LABELING, sequence_length=500)

/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:22: UserWarning: bert4keras.bert has been renamed as bert4keras.models.
  warnings.warn('bert4keras.bert has been renamed as bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:23: UserWarning: please use bert4keras.models.
  warnings.warn('please use bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:44: UserWarning: bert4keras.tokenizer has been renamed as bert4keras.tokenizers.
  warnings.warn('bert4keras.tokenizer has been renamed as bert4keras.tokenizers.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:45: UserWarning: please use bert4keras.tokenizers.
  warnings.warn('please use bert4keras.tokenizers.')


In [ ]:
from kashgari.tasks.labeling import BiLSTM_CRF_Model
from keras.callbacks import ModelCheckpoint

model = BiLSTM_CRF_Model(bert_embed)

# Saving the best model only
# filepath="ner-bert-bilstm-crf-model.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]

history = model.fit(train_x, train_y, x_validate=dev_x, y_validate=dev_y, epochs=20, batch_size=64)
model.save("ner-bert-bilstm-crf-model.h5")

/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:22: UserWarning: bert4keras.bert has been renamed as bert4keras.models.
  warnings.warn('bert4keras.bert has been renamed as bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:23: UserWarning: please use bert4keras.models.
  warnings.warn('please use bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:44: UserWarning: bert4keras.tokenizer has been renamed as bert4keras.tokenizers.
  warnings.warn('bert4keras.tokenizer has been renamed as bert4keras.tokenizers.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:45: UserWarning: please use bert4keras.tokenizers.
  warnings.warn('please use bert4keras.tokenizers.')


Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 500)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 500)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 500, 768), ( 16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 500, 768)     1536        Input-Segment[0][0]              
___________________________________________________________________________________________

In [ ]:
# pred和test部分长度不同，进行填充
test_pred = model.predict(test_x) 
count1, count2 = 0, 0
pred_final = []
for test_tag, pred_tag in zip(test_y,test_pred): #将对象中对应的元素打包成一个个元组,然后返回由这些元组组成的列表。
  if len(test_tag) > len(pred_tag): # 填充predtag
    count1 += 1
    pred_final.append(pred_tag+['O']*(len(test_tag)-len(pred_tag)))
  elif len(test_tag) < len(pred_tag): # 不存在这种情况
    count2 += 1
  else:
    pred_final.append(pred_tag)
print(count1, count2)
print(len(test_pred), len(test_y))
print(pred_final[0])

25 0
787 787
['B-disease', 'I-disease', 'I-disease', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-body', 'I-body', 'B-subject', 'I-subject', 'I-subject', 'I-subject', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
import kashgari
from kashgari.tasks.labeling import BiLSTM_CRF_Model

hyper = BiLSTM_CRF_Model.get_default_hyper_parameters() # 超参数
print(hyper)

{'layer_blstm': {'units': 128, 'return_sequences': True}, 'layer_dense': {'units': 64, 'activation': 'tanh'}}


In [ ]:
# 直接加载模型
import kashgari
from kashgari.tasks.labeling import BiLSTM_CRF_Model

hyper = BiLSTM_CRF_Model.get_default_hyper_parameters() # 超参数
print(hyper)

# loaded_model = BiLSTM_CRF_Model(bert_embed)
# loaded_model.load_weights('ner-bert-bilstm-crf-model.hdf5')
loaded_model = kashgari.utils.load_model('ner-bert-bilstm-crf-model.h5')
test_pred = loaded_model.predict(test_x)
count1, count2 = 0, 0
pred_final = []
for test_tag, pred_tag in zip(test_y,test_pred):
  if len(test_tag) > len(pred_tag): # 填充predtag
    count1 += 1
    pred_final.append(pred_tag+['O']*(len(test_tag)-len(pred_tag)))
  elif len(test_tag) < len(pred_tag): # 不存在这种情况
    count2 += 1
  else:
    pred_final.append(pred_tag)
print(count1, count2)
print(len(test_pred), len(test_y))
print(pred_final[0])
print(test_y[0])

In [ ]:
# 分析test前5个text
for i in range(5):
  print("pred"+str(i),pred_final[i])
  print("test"+str(i),test_y[i])

pred0 ['B-disease', 'I-disease', 'I-disease', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-body', 'I-body', 'B-subject', 'I-subject', 'I-subject', 'I-subject', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
test0 ['B-disease', 'I-disease', 'I-disease', 'O', 'O', 'O', 'O', 'O', 'O', 'B-body', 'I-body', 'I-body', 'I-body', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-subject', 'I-subject', 'O', 'O', 'B-body', 'I-body', 'B-subject', 'I-subject', 'I-subject', 'I-subject', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
pred1 ['B-body', 'I-body', 'I-body', 'I-body', 'I-body', 'I-body', 'B-subject', 'I-subject', 'I-subject', 'I-subject', 'O', 'B-body', 'I-body', 'I-b

In [ ]:
print(test_y[:5])

[['B-disease', 'I-disease', 'I-disease', 'O', 'O', 'O', 'O', 'O', 'O', 'B-body', 'I-body', 'I-body', 'I-body', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-subject', 'I-subject', 'O', 'O', 'B-body', 'I-body', 'B-subject', 'I-subject', 'I-subject', 'I-subject', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-body', 'I-body', 'I-body', 'I-body', 'I-body', 'I-body', 'I-body', 'I-body', 'I-body', 'B-subject', 'I-subject', 'I-subject', 'I-subject', 'I-subject', 'I-subject', 'I-subject', 'I-subject', 'I-subject', 'I-subject', 'I-subject', 'O', 'B-body', 'O', 'B-body', 'O', 'B-body', 'I-body', 'I

In [ ]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

#展示分析5个案例
test_pred_labels = [['B-disease', 'I-disease', 'I-disease', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-body', 'I-body', 'B-subject', 'I-subject', 'I-subject', 'I-subject', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
['B-body', 'I-body', 'I-body', 'I-body', 'I-body', 'I-body', 'B-subject', 'I-subject', 'I-subject', 'I-subject', 'O', 'B-body', 'I-body', 'I-body', 'I-body', 'I-body', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-body', 'I-body', 'I-body', 'I-body', 'B-subject', 'I-subject', 'O', 'O', 'O', 'O', 'O', 'O', 'B-body', 'I-body', 'I-body', 'I-body', 'I-body', 'I-body', 'O', 'B-body', 'I-body', 'O', 'B-body', 'I-body', 'B-subject', 'I-subject', 'O', 'O', 'O', 'B-subject', 'I-subject', 'I-subject', 'O', 'O', 'O', 'O', 'O', 'B-body', 'I-body', 'I-body', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-body', 'B-subject', 'I-subject', 'O', 'B-body', 'I-body', 'B-subject', 'I-subject', 'I-subject', 'I-subject', 'O', 'B-body', 'I-body', 'B-subject', 'I-subject', 'O', 'O', 'O', 'O', 'O', 'B-body', 'I-body', 'I-body', 'I-body', 'B-decorate', 'I-decorate', 'I-decorate', 'B-subject', 'I-subject', 'O', 'B-subject', 'I-subject', 'O', 'O', 'O', 'B-subject', 'I-subject', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-item', 'I-item', 'I-item', 'I-item', 'I-item', 'I-item', 'I-item', 'B-subject', 'I-subject', 'O'],
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-body', 'I-body', 'B-subject', 'I-subject', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-subject', 'I-subject', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
['B-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'O', 'O', 'O', 'O', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'O', 'O', 'O', 'O', 'O', 'O', 'B-body', 'I-body', 'B-body', 'O', 'B-body', 'I-body', 'B-subject', 'I-disease', 'I-disease', 'O', 'O', 'O', 'B-subject', 'I-subject', 'I-subject', 'I-subject', 'O', 'B-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'O', 'B-disease', 'I-disease', 'I-disease', 'O', 'B-disease', 'I-disease', 'I-disease', 'I-disease', 'O', 'B-disease', 'I-disease', 'I-disease', 'I-disease', 'I-disease', 'O', 'B-body', 'I-body', 'I-body', 'B-subject', 'I-subject', 'I-subject', 'I-subject', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-body', 'I-body', 'O', 'B-body', 'I-body', 'B-subject', 'I-subject', 'B-body', 'I-body', 'B-subject', 'I-subject', 'B-subject', 'I-subject', 'I-subject', 'I-subject', 'I-subject', 'I-subject', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-item', 'I-item', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-subject', 'I-subject', 'I-subject', 'O', 'O', 'O']]
for i in range(5):
    print("******************第%d个数据******************"%i)
    print("text: ","".join(test_x[i]))
    print("true label: ",test_y[i])
    print("pred label: ",test_pred_labels[i])
    result=[(a,b,c) for a,b,c in zip(test_x[i],test_y[i],test_pred_labels[i])]
    print(result)
    index=0
    for a,b,c in result:
        if b==c and b!="O":
            print(index,a,b,c)
        index+=1
    print("F1-score: {:.2%}".format(f1_score(test_y[i], test_pred_labels[i])))
    print(classification_report(test_y[i], test_pred_labels[i],digits=4))

******************第0个数据******************
text:  GSD依其所缺陷的酶12型，多数属分解代谢上的缺陷，使糖原异常堆积。除GSDⅨb型为X连锁隐性遗传外，其余都是常染色体隐性遗传性疾病。
true label:  ['B-disease', 'I-disease', 'I-disease', 'O', 'O', 'O', 'O', 'O', 'O', 'B-body', 'I-body', 'I-body', 'I-body', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-subject', 'I-subject', 'O', 'O', 'B-body', 'I-body', 'B-subject', 'I-subject', 'I-subject', 'I-subject', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
pred label:  ['B-disease', 'I-disease', 'I-disease', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-body', 'I-body', 'B-subject', 'I-subject', 'I-subject', 'I-subject', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[('G', 'B-disease

In [ ]:
wronglist = []
for i in range(787):
  if len(pred_final[i]) != len(test_y[i]):
    wronglist.append([len(pred_final[i]), len(test_y[i])])
print(wronglist)
print(len(wronglist))
print(pred_final[0])
print(test_y[0])
ylen = [len(_)for _ in test_y]
predlen = [len(_) for _ in pred_final]
print(ylen)
print(predlen)

[]
0
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-subject', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['B-disease', 'I-disease', 'I-disease', 'O', 'O', 'O', 'O', 'O', 'O', 'B-body', 'I-body', 'I-body', 'I-body', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-subject', 'I-subject', 'O', 'O', 'B-body', 'I-body', 'B-subject', 'I-subject', 'I-subject', 'I-subject', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[67, 136, 213, 166, 18, 85, 94, 100, 103, 111, 196, 80, 158, 121, 232, 123, 44, 80, 61, 96, 54, 49, 103, 24, 227, 166, 77, 261, 78, 204, 135, 64, 35, 181, 223, 33, 136, 127, 65, 202, 122, 127, 56, 333, 240, 107, 1

In [ ]:
! pip install seqeval

In [ ]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
print("F1-score: {:.2%}".format(f1_score(test_y, pred_final)))
print(classification_report(test_y, pred_final))

F1-score: 40.76%
           precision    recall  f1-score   support

  subject       0.42      0.41      0.42      3011
frequency       0.50      0.12      0.19        50
  disease       0.32      0.40      0.36       610
     item       0.24      0.18      0.20       177
     body       0.41      0.50      0.45      1448
 decorate       0.47      0.29      0.36       632

micro avg       0.41      0.41      0.41      5928
macro avg       0.41      0.41      0.40      5928



In [ ]:
! pip install sklearn_crfsuite

     |████████████████████████████████| 747kB 6.5MB/s 


In [ ]:
from  sklearn_crfsuite.metrics import flat_classification_report  
report = flat_classification_report(y_pred=pred_final, y_true=test_y)
print(report)

              precision    recall  f1-score   support

      B-body       0.45      0.55      0.49      1445
  B-decorate       0.52      0.32      0.40       632
   B-disease       0.36      0.43      0.39       610
 B-frequency       0.50      0.12      0.19        50
      B-item       0.34      0.20      0.25       177
   B-subject       0.47      0.46      0.47      2931
      I-body       0.44      0.56      0.49      2435
  I-decorate       0.53      0.30      0.38      1260
   I-disease       0.38      0.49      0.42      2496
 I-frequency       0.50      0.06      0.11       101
      I-item       0.31      0.16      0.21       380
   I-subject       0.53      0.37      0.44      7269
           O       0.89      0.91      0.90     87901

    accuracy                           0.82    107687
   macro avg       0.48      0.38      0.40    107687
weighted avg       0.82      0.82      0.82    107687

